In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_set = pd.read_csv('../input/saftey_efficay_myopiaTrain.csv')
test_set = pd.read_csv('../input/saftey_efficay_myopiaTest.csv')

In [ ]:
train_set = train_set.dropna(how='all')

In [ ]:
catigorial_columns = ['D_L_Sex', 'D_L_Eye', 
                      'D_L_Dominant_Eye', 'Pre_L_Contact_Lens', 
                      'T_L_Laser_Type', 'T_L_Treatment_Type', 
                      'T_L_Cust._Ablation', 'T_L_Micro', 'T_L_Head', 
                      'T_L_Therapeutic_Cont._L.', 'T_L_Epith._Rep.', 'Class']
numeric_columns = train_set.columns[~train_set.columns.isin(catigorial_columns)]

In [ ]:
train_set = train_set.apply(lambda x: x.replace([' ', '', 'NaN', 'nan', 'None'], np.nan))
test_set = train_set.apply(lambda x: x.replace([' ', '', 'NaN', 'nan', 'None'], np.nan))

In [ ]:
# preprocess-fillna for catiforial values using madien values
dict = defaultdict(LabelEncoder)
    
for col_name in catigorial_columns:
    train_set[col_name].fillna(train_set[col_name].value_counts().index[0], inplace=True)
    if col_name != 'Class':
        test_set[col_name].fillna(train_set[col_name].value_counts().index[0], inplace=True)

for col_name in catigorial_columns:
    train_set[col_name] = dict[col_name].fit_transform(train_set[col_name])
    if col_name != 'Class':
        test_set[col_name] = dict[col_name].fit_transform(test_set[col_name])


In [ ]:
# preprocess-fillna for numeric values using mean values

for col_name in numeric_columns:
    train_set[col_name].fillna(train_set[col_name].mean(), inplace=True)
    test_set[col_name].fillna(train_set[col_name].mean(), inplace=True)

In [ ]:
train_set.loc[:,catigorial_columns].isna().sum()

In [ ]:
    x_train = train_set.iloc[:, :-1]
    y_train = train_set.iloc[:, -1]

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, max_depth=5)
rf_model.fit(x_train, y_train)

In [ ]:
pred = rf_model.predict(x_train)

In [ ]:
rf_score = sum(i == 0 for i in (pred-y_train))/len(pred)
print(rf_score)